In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from print2log import redirect_to_logger
from ftp_utils import create_connect
from typing import Optional
import mylogger
import logging
from pathlib import Path
from ftplib import FTP

logger = logging.getLogger()

In [45]:
FTP_HOST = "localhost"
FTP_USER = "ITuser"
FTP_PSWD = "1234"

TARGET_PATH = "./result/work"
TARGET_TYPE = "JPG"

In [46]:
def download_dir(ftp: FTP, source: str, target: str | Path, target_type: str = TARGET_TYPE):
    lst_file = [
        filename
        for filename, attr in ftp.mlsd(source)
        if attr["type"] == "file" and filename.upper().endswith(f".{target_type}")
    ]
    logger.info(f"Get File List from [{source}]: {lst_file}")

    local_path = Path(target) / source
    local_path.mkdir(exist_ok=True, parents=True)
    for filename in lst_file:
        file_path = local_path / filename
        with open(file_path, "wb") as fp:
            ftp.retrbinary(f"RETR {source}/{filename}", fp.write)


@redirect_to_logger(msg="FTP")
def get_data_from_scc(
    host: str,
    user: str,
    pswd: str,
    toolId: str,
    local_path: str | Path,
    st: Optional[str] = None,
    et: Optional[str] = None,
):
    root = f"CDSC/{toolId}"

    with create_connect(host, user, pswd) as ftp:
        logger.info(f"Move to Directory: [{root}]")
        ftp.cwd(root)
        lst_dir = [name for name, attr in ftp.mlsd() if attr["type"] == "dir"]
        logger.info(f"Found Folder List: {lst_dir}")

        if isinstance(st, str) and isinstance(et, str):
            logger.info(f"Search Folder between [{st}] ~ [{et}]")
            lst_download_dir = [dir for dir in lst_dir if dir <= et and dir >= st]
        else:
            lst_download_dir = [max(lst_dir)]

        logger.info(f"Download Folder List: {lst_download_dir}")
        for dir in lst_download_dir:
            download_dir(ftp, dir, local_path)


get_data_from_scc(
    host=FTP_HOST,
    user=FTP_USER,
    pswd=FTP_PSWD,
    toolId="TOOL1",
    local_path=TARGET_PATH,
    st="20251215182035",
    et="20251215182040",
)

2025-12-15 04:01:26,841 - root - INFO - Try to Use FTPS
2025-12-15 04:01:26,855 - root - INFO - Create FTPS Success
2025-12-15 04:01:26,856 - root - INFO - Move to Directory: [CDSC/TOOL1]
2025-12-15 04:01:26,859 - root - INFO - Found Folder List: ['20251215182035', '20251215182040', '20251215182045']
2025-12-15 04:01:26,860 - root - INFO - Search Folder between [20251215182035] ~ [20251215182040]
2025-12-15 04:01:26,860 - root - INFO - Download Folder List: ['20251215182035', '20251215182040']
2025-12-15 04:01:26,863 - root - INFO - Get File List from [20251215182035]: ['01.jpg', '02.jpg', '03.jpg', '04.jpg']
2025-12-15 04:01:26,887 - root - INFO - Get File List from [20251215182040]: ['01.jpg', '02.jpg', '03.jpg', '04.jpg']
